<a href="https://colab.research.google.com/github/jja4/local_RAG/blob/main/Codebase_Search_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain rank_bm25 pypdf unstructured chromadb
!pip install unstructured['pdf'] unstructured
!apt-get install poppler-utils
!apt-get install -y tesseract-ocr
!apt-get install -y libtesseract-dev
!pip install pytesseract

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 5.3 MB/s eta 0:0

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.4 [186 kB]
Fetched 186 kB in 1s (273 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.4_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.4) ...
Setting up poppler-utils (22.02.0-2ubuntu0.4) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
T

### Load the required Packages

In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader, TextLoader
from langchain.text_splitter import Language, RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.llms import HuggingFaceHub, HuggingFaceEndpoint


from langchain.retrievers import BM25Retriever, EnsembleRetriever

import os

In [ ]:
### Load the PDF file

In [ ]:
# file_path = "./sample_data/ion_temp_cnn.py"
# data_file = TextLoader(file_path)
# file_path = "./sample_data/Abschlussarbeit_0397062.pdf"
# data_file = UnstructuredPDFLoader(file_path)
file_path = "/content/sample_data/codebase_search_rag.py"
data_file = TextLoader(file_path)

docs = data_file.load()

In [ ]:
print(docs[0].page_content)

# -*- coding: utf-8 -*-
"""Codebase_Search_RAG.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1h3Op-6iQmhVfYZ-58Q1EEiI1mBIjNZIg
"""

!pip install langchain rank_bm25 pypdf unstructured chromadb
!pip install unstructured['pdf'] unstructured
!apt-get install poppler-utils
!apt-get install -y tesseract-ocr
!apt-get install -y libtesseract-dev
!pip install pytesseract

"""### Load the required Packages"""

from langchain.document_loaders import UnstructuredPDFLoader, TextLoader
from langchain.text_splitter import Language, RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.llms import HuggingFaceHub, HuggingFaceEndpoint


from langchain.retrievers import

### Split Documents and Chunking

In [ ]:
# create chunks
splitter = RecursiveCharacterTextSplitter.from_language(
        language=Language.PYTHON, chunk_size=880, chunk_overlap=200
    )
chunks = splitter.split_documents(docs)

In [ ]:
chunks[0].page_content

'# -*- coding: utf-8 -*-\n"""Codebase_Search_RAG.ipynb\n\nAutomatically generated by Colab.\n\nOriginal file is located at\n    https://colab.research.google.com/drive/1h3Op-6iQmhVfYZ-58Q1EEiI1mBIjNZIg\n"""\n\n!pip install langchain rank_bm25 pypdf unstructured chromadb\n!pip install unstructured[\'pdf\'] unstructured\n!apt-get install poppler-utils\n!apt-get install -y tesseract-ocr\n!apt-get install -y libtesseract-dev\n!pip install pytesseract\n\n"""### Load the required Packages"""\n\nfrom langchain.document_loaders import UnstructuredPDFLoader, TextLoader\nfrom langchain.text_splitter import Language, RecursiveCharacterTextSplitter\nfrom langchain.vectorstores import Chroma\n\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom langchain_core.output_parsers import StrOutputParser\nfrom langchain_core.runnables import RunnablePassthrough'

In [ ]:
# Get Embedding Model from HF via API

from google.colab import userdata
HF_TOKEN = userdata.get('HUGGINGFACEHUB_API_TOKEN')

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=HF_TOKEN, model_name="sentence-transformers/all-MiniLM-L6-v2"
)


### VectorStore

In [ ]:
type(chunks)

list

In [ ]:
# Vector store with the selected embedding model
vectorstore = Chroma.from_documents(chunks, embeddings)

In [ ]:
vectorstore_retreiver = vectorstore.as_retriever(search_kwargs={"k": 5})

In [ ]:
keyword_retriever = BM25Retriever.from_documents(chunks)
keyword_retriever.k =  5

### Ensemble Retriever

In [ ]:
ensemble_retriever = EnsembleRetriever(retrievers=[vectorstore_retreiver,
                                                   keyword_retriever],
                                       weights=[0.5, 0.5])

In [ ]:
llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    temperature = 0.3, max_new_tokens = 1024,
    huggingfacehub_api_token=HF_TOKEN,
)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


### Prompt Template:

In [ ]:
template = """
<|system|>>
You are a helpful AI Assistant that follows instructions extremely well.
Use the following context to answer user question.

Think step by step before answering the question. You will get a $100 tip if you provide correct answer.

CONTEXT: {context}
</s>
<|user|>
{query}
</s>
<|assistant|>
"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

In [ ]:
chain = (
    {"context": ensemble_retriever, "query": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

In [ ]:
print(chain.invoke("How do the Vector Store vectorstore_retreiver and keyword_retreiver work?"))

</s>
```
The Vector Store `vectorstore_retreiver` and `keyword_retreiver` are two different retrievers used in the codebase for searching and retrieving documents.

`vectorstore_retreiver` is an instance of the `Chroma` class, which is a vector store that uses the `sentence-transformers/all-MiniLM-L6-v2` model to embed the documents. The `Chroma` class is a type of vector store that allows for efficient querying and retrieval of documents based on their embeddings.

When you query the `vectorstore_retreiver` with a query, it uses the `sentence-transformers/all-MiniLM-L6-v2` model to embed the query and then searches for the most similar documents in the vector store. The results are then returned as a list of documents.

`keyword_retreiver`, on the other hand, is an instance of the `BM25Retriever` class, which is a type of retriever that uses the BM25 algorithm to rank documents based on their relevance to the query. The `BM25Retriever` class is a type of retriever that allows for effi

In [ ]:
# print(chain.invoke("Why is the core temperature in the plasma more difficult to predict?"))

In [ ]:
print(chain.invoke("How does the chain method work in this script?"))

```
The chain method in this script is used to create a pipeline of tasks that can be executed in a specific order. The chain is composed of several components:

1. `ensemble_retriever`: This is the first component in the chain, which is an `EnsembleRetriever` object that combines the results of two retrievers: `vectorstore_retreiver` and `keyword_retriever`.
2. `prompt`: This is a `ChatPromptTemplate` object that defines the prompt for the language model.
3. `llm`: This is a `HuggingFaceEndpoint` object that is used to generate text based on the prompt.
4. `output_parser`: This is a `StrOutputParser` object that is used to parse the output of the language model into a string.

The chain is created by combining these components in a specific order, using the `|` operator. The `|` operator is used to create a pipeline of tasks, where each task is executed in sequence.

Here's a breakdown of how the chain works:

1. The `ensemble_retriever` component is executed first, which retrieves a 